# Import des données de test sur la période du 1er Janvier 2024 au 26 février 2025

In [15]:
import pandas as pd

# Chemin du fichier sur Kaggle
chemin = "/kaggle/input/crypto/df_export.csvtest_df.csv"

# Charger le DataFrame
df = pd.read_csv(chemin)

# Afficher les premières lignes pour vérifier
print(df.head())

             timestamp     open     high      low    close     volume  rsi  \
0  2024-01-01 00:00:00  2281.87  2287.84  2281.27  2287.84   910.5701  NaN   
1  2024-01-01 00:05:00  2287.83  2291.83  2287.06  2290.77  1652.9290  NaN   
2  2024-01-01 00:10:00  2290.77  2296.97  2290.76  2296.71  1829.5259  NaN   
3  2024-01-01 00:15:00  2296.71  2297.00  2292.36  2292.36   890.8524  NaN   
4  2024-01-01 00:20:00  2292.37  2295.87  2292.30  2294.67   763.7046  NaN   

       macd  macd_signal  atr  volatility_20d  roc_5  roc_10  
0  0.000000     0.000000  NaN             NaN    NaN     NaN  
1  0.065737     0.036521  NaN             NaN    NaN     NaN  
2  0.267961     0.131373  NaN             NaN    NaN     NaN  
3  0.196315     0.153372  NaN             NaN    NaN     NaN  
4  0.245170     0.180680  NaN             NaN    NaN     NaN  


/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


# On shift le close et on utiliseras celui la pour simuler la période de retard mais le portefeuille évoluera selon le nom shifter

On calcul nos indicateurs sur le close et lors de nos trade on les passeras sur le close_shifted pour simulé une période de retard

In [16]:
sma_short = 5  # Période courte
sma_long = 190  # Période longue

# Calcul des Moyennes Mobiles sur close_shifted
df["SMA_Short"] = df["close"].rolling(window=sma_short).mean()
df["SMA_Long"] = df["close"].rolling(window=sma_long).mean()

# Détecter les Signaux d'Achat/Vente
df["Signal"] = 0
df.loc[df["SMA_Short"] > df["SMA_Long"], "Signal"] = 1  # Achat
df.loc[df["SMA_Short"] < df["SMA_Long"], "Signal"] = -1  # Vente

# Vérifier les premières lignes
print(df[["timestamp", "close", "SMA_Short", "SMA_Long", "Signal"]].head())


             timestamp    close  SMA_Short  SMA_Long  Signal
0  2024-01-01 00:00:00  2287.84        NaN       NaN       0
1  2024-01-01 00:05:00  2290.77        NaN       NaN       0
2  2024-01-01 00:10:00  2296.71        NaN       NaN       0
3  2024-01-01 00:15:00  2292.36        NaN       NaN       0
4  2024-01-01 00:20:00  2294.67    2292.47       NaN       0


/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)
/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [17]:
df = df.dropna().reset_index(drop=True)


In [18]:
print(df)

                  timestamp     open     high      low    close     volume  \
0       2024-01-01 15:45:00  2317.37  2318.34  2312.00  2313.39  1283.2452   
1       2024-01-01 15:50:00  2313.39  2314.59  2311.72  2313.72   659.4002   
2       2024-01-01 15:55:00  2313.72  2317.29  2312.00  2314.70  1296.1569   
3       2024-01-01 16:00:00  2314.69  2315.74  2312.62  2312.63  1205.0342   
4       2024-01-01 16:05:00  2312.63  2317.33  2312.52  2316.83  1270.3909   
...                     ...      ...      ...      ...      ...        ...   
121343  2025-02-25 23:40:00  2498.49  2500.72  2494.17  2497.55   791.4884   
121344  2025-02-25 23:45:00  2497.50  2501.91  2495.75  2499.71   777.7942   
121345  2025-02-25 23:50:00  2499.71  2500.40  2494.68  2494.82   933.5947   
121346  2025-02-25 23:55:00  2494.82  2498.29  2494.70  2495.70   946.6197   
121347  2025-02-26 00:00:00  2495.69  2499.67  2494.30  2496.31   975.6504   

              rsi      macd  macd_signal       atr  volatility_

Stratégie 2 : SMA sans indicateurs

# 1. Application de la stratégie 1 (calcul hors frais et frais 0.05% achat & revente)

### La stratégie 1 est juste l'achat revente selon les moyenne mobiles optimum calculé sur le train

In [19]:
import matplotlib.pyplot as plt

# Paramètres de backtest
initial_capital = 5000
capital = initial_capital
capital_frais = initial_capital  # Capital avec frais
position = 0  # Nombre d'ETH détenus
position_frais = 0  # Nombre d'ETH détenus avec frais

# Supprimer les lignes avec des valeurs manquantes dans les colonnes essentielles

# Initialiser les colonnes pour le capital
df["Stratégie2"] = capital
df["Stratégie2_Frais"] = capital_frais  # Avec frais

# Backtest de la stratégie
for i in range(1, len(df)):
    signal = df["Signal"].iloc[i]
    close_price = df["close"].iloc[i]  # Utilisation de close_shifted

    # 🚀 Achat sans frais
    if signal == 1 and position == 0 and capital >= close_price:
        position = 1
        capital -= close_price

    # 💰 Vente sans frais
    elif signal == -1 and position == 1:
        capital += position * close_price
        position = 0

    # 📉 Achat avec frais (on paie 0.05% en plus)
    if signal == 1 and position_frais == 0 and capital_frais >= close_price * 1.0005:
        position_frais = 1
        capital_frais -= close_price * 1.0005  # Achat avec frais

    # 📈 Vente avec frais (on vend à 0.05% en moins)
    elif signal == -1 and position_frais == 1:
        capital_frais += position_frais * close_price * 0.9995  # Vente avec frais
        position_frais = 0

    # Mise à jour du capital total (cash + valeur de l'ETH détenu)
    df.at[df.index[i], "Stratégie1"] = capital + (position * close_price)
    df.at[df.index[i], "Stratégie1_Frais"] = capital_frais + (position_frais * close_price)



# 2. Application de la stratégie 2 (calcul hors frais et frais 0.05% achat & revente)

### Strat 2 est la même que la 1 avec des conditions sur certains indicateurs pris au hasard

### cette stratégie passe beaucoup de temps sans passer d'odre (pas forcément opti mais non backtester donc paramètre fixer avec une demande à un bot au départ et non changé pour optimisation)

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Paramètres de backtest
initial_capital = 5000
capital = initial_capital
capital_frais = initial_capital
position = 0  # Nombre d'ETH détenus
position_frais = 0

# Périodes des moyennes mobiles
MM_courte = 5
MM_longue = 190  

# Calcul des Moyennes Mobiles sur close_shifted
df["MM_courte"] = df["close"].rolling(window=MM_courte).mean()
df["MM_longue"] = df["close"].rolling(window=MM_longue).mean()

# Calcul des indicateurs techniques
def compute_rsi(series, window=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def compute_macd(series, short_window=12, long_window=26, signal_window=9):
    short_ema = series.ewm(span=short_window, min_periods=1).mean()
    long_ema = series.ewm(span=long_window, min_periods=1).mean()
    macd = short_ema - long_ema
    macd_signal = macd.ewm(span=signal_window, min_periods=1).mean()
    return macd, macd_signal

def compute_atr(data, window=14):
    high_low = data["high"] - data["low"]
    high_close = np.abs(data["high"] - data["close"].shift())
    low_close = np.abs(data["low"] - data["close"].shift())
    tr = pd.DataFrame({"high_low": high_low, "high_close": high_close, "low_close": low_close}).max(axis=1)
    return tr.rolling(window=window).mean()

# Ajouter les indicateurs techniques
df["rsi"] = compute_rsi(df["close"], window=14)
df["macd"], df["macd_signal"] = compute_macd(df["close"])
df["atr"] = compute_atr(df)
df["volatility_20d"] = df["close"].pct_change().rolling(20).std()
df["roc_5"] = df["close"].pct_change(5)
df["roc_10"] = df["close"].pct_change(10)

# Définir les signaux dans la colonne Signal
df["Signal"] = 0  

# Condition d'achat
df.loc[
    (df["MM_courte"] > df["MM_longue"]) &  
    (df["rsi"] < 40) &  
    (df["macd"] > df["macd_signal"]),  
    "Signal"
] = 1  

# Condition de vente
df.loc[
    (df["MM_courte"] < df["MM_longue"]) &  
    (df["rsi"] > 60) &  
    (df["macd"] < df["macd_signal"]),  
    "Signal"
] = -1  

# Supprimer les lignes avec des valeurs manquantes
df = df.dropna(subset=["Signal", "close", "timestamp", "rsi", "macd", "macd_signal"])

# Initialiser les colonnes de capital
df["Strat_2"] = capital
df["Strat_2_Frais"] = capital_frais

frais = 0.0005  # 0.05% de frais

# Backtest de la stratégie
for i in range(1, len(df)):
    signal = df["Signal"].iloc[i]
    close_price = df["close"].iloc[i]
    
    # Achat sans frais
    if signal == 1 and position == 0 and capital >= close_price:
        position = 1
        capital -= close_price
    
    # Vente sans frais
    elif signal == -1 and position == 1:
        capital += position * close_price
        position = 0
    
    # Achat avec frais
    if signal == 1 and position_frais == 0 and capital_frais >= close_price * (1 + frais):
        position_frais = 1
        capital_frais -= close_price * (1 + frais)
    
    # Vente avec frais
    elif signal == -1 and position_frais == 1:
        capital_frais += position_frais * close_price * (1 - frais)
        position_frais = 0
    
    # Mise à jour du capital total (cash + valeur de l'ETH détenu)
    df.at[df.index[i], "Strat_2"] = capital + (position * close_price)
    df.at[df.index[i], "Strat_2_Frais"] = capital_frais + (position_frais * close_price)

/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)
/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)
/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)
/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)
/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)
/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)
<ipython-input-20-9a46329bc8a7>:105: FutureWarning: Setting an item of incompatible d

# Benchmark (achat d'un eth et keep jusqu'à échéance)

In [21]:
import matplotlib.pyplot as plt

# Paramètres de backtest
initial_capital = 5000
capital = initial_capital
capital_frais = initial_capital  # Capital avec frais
position = 0  # Nombre d'ETH détenus
position_frais = 0  # Nombre d'ETH détenus avec frais

# Initialiser les colonnes pour le capital
df["BH"] = capital
df["BH_Frais"] = capital_frais  # Avec frais

# Acheter un ETH au tout début
initial_price = df["close"].iloc[0]  # Prix d'achat au début

# Achat sans frais
capital -= initial_price
position = 1  # Acheter 1 ETH

# Achat avec frais (on paie 0.05% en plus)
capital_frais -= initial_price * 1.0005
position_frais = 1  # Acheter 1 ETH

# Backtest de la stratégie Buy & Hold
for i in range(1, len(df)):
    close_price = df["close"].iloc[i]
    
    # Mise à jour du capital total (cash + valeur de l'ETH détenu)
    df.at[df.index[i], "BH"] = capital + (position * close_price)
    df.at[df.index[i], "BH_Frais"] = capital_frais + (position_frais * close_price)

# 📊 Affichage du graphique des profits et pertes
#plt.figure(figsize=(12, 6))
#plt.plot(df["timestamp"], df["BH"], label="Buy & Hold (Sans frais)", color="blue")
#plt.plot(df["timestamp"], df["BH_Frais"], label="Buy & Hold (Avec frais 0.05%)", color="red")
#plt.xlabel("Date")
#plt.ylabel("Capital (€)")
#plt.title("Évolution du Capital avec la Stratégie Buy & Hold")
#plt.legend()
#plt.grid()
#plt.show()


<ipython-input-21-2bbfdd8b817e>:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '4998.53' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[df.index[i], "BH"] = capital + (position * close_price)
<ipython-input-21-2bbfdd8b817e>:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '4997.37432' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[df.index[i], "BH_Frais"] = capital_frais + (position_frais * close_price)


In [22]:
print(df)

                  timestamp     open     high      low    close     volume  \
13      2024-01-01 16:50:00  2311.61  2313.35  2310.86  2311.36   440.1587   
14      2024-01-01 16:55:00  2311.36  2311.39  2309.40  2309.89   533.8544   
15      2024-01-01 17:00:00  2309.89  2312.09  2306.48  2309.53  1362.1103   
16      2024-01-01 17:05:00  2309.53  2312.34  2309.53  2311.84   416.0986   
17      2024-01-01 17:10:00  2311.83  2316.51  2311.83  2315.17   998.0475   
...                     ...      ...      ...      ...      ...        ...   
121343  2025-02-25 23:40:00  2498.49  2500.72  2494.17  2497.55   791.4884   
121344  2025-02-25 23:45:00  2497.50  2501.91  2495.75  2499.71   777.7942   
121345  2025-02-25 23:50:00  2499.71  2500.40  2494.68  2494.82   933.5947   
121346  2025-02-25 23:55:00  2494.82  2498.29  2494.70  2495.70   946.6197   
121347  2025-02-26 00:00:00  2495.69  2499.67  2494.30  2496.31   975.6504   

              rsi      macd  macd_signal       atr  ...  Straté

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


## Export pour analyse des résultats

In [23]:
df.to_csv('/kaggle/working/df.csv', index=False)

# Bot d'achat et vente en fonction du signal /!\ Hypothétique et non tester car pas de compte Binance, ni de Capital suffisant

In [ ]:
import time
import pandas as pd
import requests
from datetime import datetime, timedelta

# 🔥 Paramètres du bot
initial_capital = 5000  # Capital de départ en USDT
capital = initial_capital
position = 0  # ETH détenus
order_size = 1  # Taille de l'ordre d'achat (1 ETH)
trade_history = []  # Historique des trades
df = pd.DataFrame(columns=['timestamp', 'close', 'Signal'])

# 📊 Fonction pour récupérer le prix de l'ETH via l'API publique Binance
def get_eth_price():
    try:
        url = "https://api.binance.com/api/v3/ticker/price?symbol=ETHUSDT"
        response = requests.get(url)
        data = response.json()
        return float(data['price'])
    except Exception as e:
        print(f"⚠️ Erreur récupération prix : {e}")
        return None

# 📈 Calcul du RSI
def compute_rsi(series, window=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

# 📊 Calcul des moyennes mobiles (MM45 et MM190)
def calculate_moving_averages(df):
    df["MM45"] = df["close"].rolling(window=45).mean()
    df["MM190"] = df["close"].rolling(window=190).mean()
    return df

# 🔍 Génération des signaux d'achat/vente avec RSI + Moyennes Mobiles
def calculate_signals(df):
    df['rsi'] = compute_rsi(df['close'], window=14)
    df = calculate_moving_averages(df)

    df['Signal'] = 0  # Par défaut, pas de signal

    # Achat si RSI < 30 ET MM45 > MM190 (momentum haussier)
    df.loc[(df['rsi'] < 30) & (df['MM45'] > df['MM190']), 'Signal'] = 1  

    # Vente si RSI > 70 ET MM45 < MM190 (momentum baissier)
    df.loc[(df['rsi'] > 70) & (df['MM45'] < df['MM190']), 'Signal'] = -1  

    return df

# 🛒 Fonction pour exécuter les ordres virtuels
def place_order(signal, capital, position, current_price):
    if signal == 1 and position == 0 and capital >= current_price * order_size:
        # Achat
        position = 1
        capital -= current_price * order_size
        trade_history.append(f"🟢 Achat de 1 ETH à {current_price} USDT")
        print(trade_history[-1])
        return capital, position

    elif signal == -1 and position == 1:
        # Vente
        capital += current_price * order_size
        position = 0
        trade_history.append(f"🔴 Vente de 1 ETH à {current_price} USDT")
        print(trade_history[-1])
        return capital, position

    return capital, position

# 🔄 Test du bot sur 15 minutes
start_time = datetime.now()
end_time = start_time + timedelta(minutes=15)

while datetime.now() < end_time:
    current_price = get_eth_price()
    if current_price is None:
        time.sleep(60)  # Attendre 1 minute avant de réessayer en cas d'erreur API
        continue

    timestamp = pd.to_datetime('now')

    # Ajout des nouvelles données
    new_data = pd.DataFrame({'timestamp': [timestamp], 'close': [current_price]})
    df = pd.concat([df, new_data], ignore_index=True)

    # Calcul des signaux
    if len(df) > 190:  # Vérifier qu'on a assez de données pour MM190
        df = calculate_signals(df)

        # Récupérer le dernier signal
        latest_signal = df["Signal"].iloc[-1]

        # Passer un ordre si nécessaire
        capital, position = place_order(latest_signal, capital, position, current_price)

        # Mise à jour du capital
        df.at[df.index[-1], "Stratégie2"] = capital + (position * current_price)

    # Affichage des informations
    print(f"\n🕒 {timestamp} - Prix ETH : {current_price} USDT")
    print(f"💰 Capital : {capital} USDT - Position ETH : {position}")
    print(f"📈 Signal : {df['Signal'].iloc[-1] if len(df) > 190 else 'En attente (données insuffisantes)'}")

    # Pause de 5 minutes avant le prochain trade
    time.sleep(300)  

print("\n✅ Test terminé ! Résumé des trades exécutés :")
for trade in trade_history:
    print(trade)

print(f"\n💰 Capital final après 15 min : {capital} USDT")


⚠️ Erreur récupération prix : 'price'
⚠️ Erreur récupération prix : 'price'
⚠️ Erreur récupération prix : 'price'
⚠️ Erreur récupération prix : 'price'
⚠️ Erreur récupération prix : 'price'
⚠️ Erreur récupération prix : 'price'
⚠️ Erreur récupération prix : 'price'
⚠️ Erreur récupération prix : 'price'
